In [33]:
import pandas as pd
import numpy as np
import time
from sklearn.model_selection import KFold

In [34]:
path = '/home/kai/data/kaggle/talkingdata/data/'
train = pd.read_csv(path + 'train_cleaned_final.csv')

# Get K Fold

In [35]:
K = 18
# kf = KFold(n_splits=K, shuffle = False)
kf = KFold(n_splits=K, shuffle = True, random_state = 233)
history_index = []
train_index = []
for h,t in kf.split(train):
    history_index.append(h)
    train_index.append(t)

### use last fold as example

In [36]:
import sys
print(sys.getsizeof(train)/ 1024 **3)

16.531696118414402


# use last fold as example

In [37]:
length_train = len(train)
df_history = train.iloc[history_index[-1]].copy()
df_train = train.iloc[train_index[-1]].copy()


In [38]:
import sys
print(sys.getsizeof(df_train)/ 1024 **3)
print(len(df_train))
print(sys.getsizeof(df_history)/ 1024 **3)
print(len(df_history))

0.994963176548481
10272438
16.914374224841595
174631452


In [39]:
df_train.columns

Index(['ip', 'app', 'device', 'os', 'channel', 'day', 'hour', 'timestamp',
       'minute', 'second', 'is_attributed', 'intesthh'],
      dtype='object')

In [40]:
orders = {}
feature_col = ['ip', 
              'app', 
              'device', 
              'os', 
              'channel',
              'day',
              'hour',
              'minute',
              'second',
              'intesthh']

# feature_col = ['ip', 
#               'app', 
#               'device', 
#               'os', 
#               'channel']
for col in feature_col:
    orders[col] = 10 ** (int(np.log(train[col].max() + 1) / np.log(10)) + 1)
def get_group(df, cols):
    """
    define an encoding method which can ganrantee the adding value will be unique.
    eg: artist_name_composer will be a combination of (artist_name,composer) and the encoding will reflect the unqiue combination of those two
    """
    group = df[cols[0]].copy()
    for col in cols[1:]:
        group = group * orders[col] + df[col]
        
    return group

import gc
del train
gc.collect()

2475

In [41]:
orders

{'app': 1000,
 'channel': 1000,
 'day': 100,
 'device': 10000,
 'hour': 100,
 'intesthh': 10,
 'ip': 1000000,
 'minute': 100,
 'os': 1000,
 'second': 100}

# count
plan 1. count from historical data  
plan 2. count from all data

In [42]:
def count(df_history, df_train, cols, target=None):
    """
    Purpose: add a new feature to training df.count the number of records for each feature combination (eg, artist_name_composer)) 
    """
    
    group = get_group(df_train, cols)
    group_all = get_group(df_history, cols)
    
    count_map = group_all.value_counts()
    
    return group.map(count_map).fillna(0)

# mean
mean P(target | feature combination)

purpose: add a new feature to training df. conditional probability P(replay (target) | feature combination (eg, artist_name_composer))
Get the conditional Probability only from historical data and apply to train data.

P(replay | X feature combination) = P( replay & X feature combination) / P (X feature combination)  
=(count(replay & X feature combination) / count(total)) / (count(X feature combination) / count(total))  
= count(replay & X feature combination) / count(X feature combination)  
= sum((replay & X feature combination)) / count(X feature combination)  
= sum((replay or not replayed & X feature combination)) / count(X feature combination)# since replay is 1, not replay is 0  
= sum( X feature combination) / count(X feature combination)  
= mean(X feature combination)  

In [43]:
def scaller(num):
    sca = 1
    while num * sca < 1:
        sca *= 10
    return sca

def mean(df_history, df_train, cols, target):
    """
    Purpose: add a new feature to training df. conditional probability P(replay (target) | feature combination (eg, artist_name_composer)) 
    """
  
    # encoding df's cols into a new series
    group = get_group(df_train, cols)
    # encoding df_history's cols into a new series
    group_history = get_group(df_history, cols)
    # get the conditional probability p(target| feature combination. eg, artist_name_composer) 
    mean_map = df_history.groupby(group_history)[target].mean()
    # mean_map: key - encoding, value - target mean
#     ### sca
#     m_min = mean_map[mean_map > 0].min()
#     sca = scaller(m_min)
#     mean_map *= sca
#     ###

    return group.map(mean_map).fillna(-1)

# reversemean
reverse mean P(feature combination | target)

In [44]:
def reversemean(df_history, df_train, cols, target):
    """
    Purpose: add a new feature to training df. conditional probability P(replay (target) | feature combination (eg, artist_name_composer)) 
    """
  
    # encoding df's cols into a new series
    group = get_group(df_train, cols)
    # encoding df_history's cols into a new series
    group_history = get_group(df_history, cols)
    # get the conditional probability p(target| feature combination. eg, artist_name_composer) 
    positive = group_history[df_history[target] == 1]
    negative = group_history[df_history[target] == 0]
    index_p = set(positive.unique())
    index_n = set(negative.unique())
    index_n.difference_update(index_p)
    map_reverse_p = positive.groupby(positive).count() / len(positive)
    map_reverse_n = pd.Series(np.zeros(len(index_n)), index=index_n)
    map_reverse = pd.concat([map_reverse_p, map_reverse_n])
    return group.map(map_reverse).fillna(-1)

# generate all cols

In [45]:
from itertools import combinations


combine_col = ['ip', 
              'app', 
              'device', 
              'os', 
              'channel', 
              'day',
              'hour',
              'intesthh']



def col_name(cols, func=None):
    if func is None:
        return '_'.join(cols)
    else:
        return '_'.join(cols) + '_' + func.__name__

counter = 0



exception_list = []
for num_col in [1,2,3,4,5]:
    for cols in combinations(combine_col, num_col):
#         for func in [count, mean]:
        for func in [reversemean]:
            feature_name = col_name(cols, func=func)
            counter += 1
            if func.__name__ == count.__name__:
                print('count function')
                df_train[feature_name] = func(df_train, df_train, cols, target='is_attributed')
            else:
                print('mean function')
                df_train[feature_name] = func(df_history, df_train, cols, target='is_attributed')
            all_str = 'all {}:   {}   \t\t\t size: {} G.'.format(counter, feature_name, sys.getsizeof(df_train)/ 1024 **3)
            print(all_str)
            with open('feature_all.txt', 'w') as text_file:
                text_file.write(all_str + '\n')


mean function
all 1:   ip_reversemean   			 size: 1.071498803794384 G.
mean function
all 2:   app_reversemean   			 size: 1.148034431040287 G.
mean function
all 3:   device_reversemean   			 size: 1.22457005828619 G.
mean function
all 4:   os_reversemean   			 size: 1.301105685532093 G.
mean function
all 5:   channel_reversemean   			 size: 1.377641312777996 G.
mean function
all 6:   day_reversemean   			 size: 1.454176940023899 G.
mean function
all 7:   hour_reversemean   			 size: 1.530712567269802 G.
mean function
all 8:   intesthh_reversemean   			 size: 1.607248194515705 G.
mean function
all 9:   ip_app_reversemean   			 size: 1.6837838217616081 G.
mean function
all 10:   ip_device_reversemean   			 size: 1.7603194490075111 G.
mean function
all 11:   ip_os_reversemean   			 size: 1.8368550762534142 G.
mean function
all 12:   ip_channel_reversemean   			 size: 1.9133907034993172 G.
mean function
all 13:   ip_day_reversemean   			 size: 1.9899263307452202 G.
mean function
all 14:   

all 100:   ip_app_os_hour_reversemean   			 size: 8.648525901138783 G.
mean function
all 101:   ip_app_os_intesthh_reversemean   			 size: 8.725061528384686 G.
mean function
all 102:   ip_app_channel_day_reversemean   			 size: 8.801597155630589 G.
mean function
all 103:   ip_app_channel_hour_reversemean   			 size: 8.878132782876492 G.
mean function
all 104:   ip_app_channel_intesthh_reversemean   			 size: 8.954668410122395 G.
mean function
all 105:   ip_app_day_hour_reversemean   			 size: 9.031204037368298 G.
mean function
all 106:   ip_app_day_intesthh_reversemean   			 size: 9.1077396646142 G.
mean function
all 107:   ip_app_hour_intesthh_reversemean   			 size: 9.184275291860104 G.
mean function
all 108:   ip_device_os_channel_reversemean   			 size: 9.260810919106007 G.
mean function
all 109:   ip_device_os_day_reversemean   			 size: 9.33734654635191 G.
mean function
all 110:   ip_device_os_hour_reversemean   			 size: 9.413882173597813 G.
mean function
all 111:   ip_device_os

all 188:   ip_device_os_hour_intesthh_reversemean   			 size: 15.383661098778248 G.
mean function
all 189:   ip_device_channel_day_hour_reversemean   			 size: 15.46019672602415 G.
mean function
all 190:   ip_device_channel_day_intesthh_reversemean   			 size: 15.536732353270054 G.
mean function
all 191:   ip_device_channel_hour_intesthh_reversemean   			 size: 15.613267980515957 G.
mean function
all 192:   ip_device_day_hour_intesthh_reversemean   			 size: 15.68980360776186 G.
mean function
all 193:   ip_os_channel_day_hour_reversemean   			 size: 15.766339235007763 G.
mean function
all 194:   ip_os_channel_day_intesthh_reversemean   			 size: 15.842874862253666 G.
mean function
all 195:   ip_os_channel_hour_intesthh_reversemean   			 size: 15.919410489499569 G.
mean function
all 196:   ip_os_day_hour_intesthh_reversemean   			 size: 15.995946116745472 G.
mean function
all 197:   ip_channel_day_hour_intesthh_reversemean   			 size: 16.072481743991375 G.
mean function
all 198:   app_d

# Saving Files

In [15]:
print(len(df_train))

10272438


In [46]:
df_train.to_csv('/home/kai/data/kaggle/talkingdata/wl/data/features/train_reversemean_combine5_0405.csv', index=False)

In [30]:
df_history.head(10)

,ip,app,device,os,channel,day,hour,timestamp,minute,second,is_attributed,intesthh
0,83230,3,1,13,379,6,14,1509978741,32,21,0,1
1,17357,3,1,19,379,6,14,1509978814,33,34,0,1
2,35810,3,1,13,379,6,14,1509978852,34,12,0,1
3,45745,14,1,13,478,6,14,1509978892,34,52,0,1
4,161007,3,1,13,379,6,14,1509978908,35,8,0,1
5,18787,3,1,16,379,6,14,1509978986,36,26,0,1
6,103022,3,1,23,379,6,14,1509979064,37,44,0,1
7,114221,3,1,19,379,6,14,1509979079,37,59,0,1
8,165970,3,1,13,379,6,14,1509979090,38,10,0,1
9,74544,64,1,22,459,6,14,1509979103,38,23,0,1
